### Sqlite database for the tables

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect

import pymysql # Not needed if mysqlclient is installed
pymysql.install_as_MySQLdb()

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date

#### Read cleaned measurements and stations CSV data

In [2]:
measurements = pd.read_csv("Resources/clean_hawaii_measurements.csv")
measurements["date"] = pd.to_datetime(measurements["date"], format="%Y-%m-%d")
measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [3]:
stations = pd.read_csv("Resources/clean_hawaii_stations.csv")
stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
measurements.dtypes

station            object
date       datetime64[ns]
prcp              float64
tobs                int64
dtype: object

In [5]:
stations.dtypes

station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object

#### Use the engine and connection string to create a database called hawaii.sqlite

In [6]:
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

#### Use `declarative_base` to create ORM classes for each table.

In [7]:
Base = declarative_base()

In [8]:
# Create ORM class for measurements
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

In [9]:
# Create ORM class for stations
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

#### Create the tables in the database using create_all

In [10]:
# Create a "Metadata" Layer That Abstracts our SQL Database
Base.metadata.create_all(engine)

# Create a Session Object to Connect to DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

Add data to the measurement table

In [11]:
# Create specific instances
for i in measurements.index:
    row = measurements.loc[i]
    instance = Measurement(station=row["station"], date=row["date"], 
                           prcp=row["prcp"], tobs=row["tobs"])
    session.add(instance)

# Commit changes to database
session.commit()

Add data to the station table

In [12]:
# Create specific instances
for i in stations.index:
    row = stations.loc[i]
    instance = Station(station=row["station"], name=row["name"], 
                           latitude=row["latitude"], longitude=row["longitude"], elevation=row["elevation"])
    session.add(instance)

# Commit changes to database
session.commit()

#### Confirm table creation

In [13]:
engine.execute("select * from measurement limit 10;").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (6, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (7, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (8, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (9, 'USC00519397', '2010-01-11', 0.01, 64.0),
 (10, 'USC00519397', '2010-01-12', 0.0, 61.0)]

In [14]:
engine.execute("select * from station limit 10;").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [15]:
# Inspect `measurement` table datatypes
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station VARCHAR(255)
date DATE
prcp FLOAT
tobs FLOAT


In [16]:
# Inspect `station` table datatypes
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station VARCHAR(255)
name VARCHAR(255)
latitude FLOAT
longitude FLOAT
elevation FLOAT
